In [1]:
import cherrypy
import json
import redis
import uuid
from redis.commands.json.path import Path
from time import time

In [2]:
REDIS_HOST = 'redis-11230.c293.eu-central-1-1.ec2.cloud.redislabs.com'
REDIS_PORT = 11230
REDIS_USER = 'default'
REDIS_PASSWORD = '2DXp9ovUfPWU8NTvfFaby0ZODJJRQnj8'
# Connect to Redis server
redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USER, password=REDIS_PASSWORD)
is_connected = redis_client.ping()
print('Redis Connected:', is_connected)

Redis Connected: True


In [3]:
#Mapping of the zones with their corresponding mac addresses

encoded_zones = {
    'magallanes': ['0xdca9047270e2', '0x2744e0dc8c68'] ,
    'acevedo' : ['0x19f57bb40d93'],
    'rivera' : [],
    'templete' : []
    }

In [4]:
class Zones(object):
    exposed = True


    def GET(self, *path, **query):
        keys = redis_client.keys('*')
        items = set()

        for key in keys:
            mac_address = key.decode()
            zone = None
            for k,v in encoded_zones.items():
                if mac_address in v:
                    zone = k
                    
            if not zone is None:
                items.add(zone)

        response_dict = {
            'zones': list(items)
        }

        response = json.dumps(response_dict)

        cherrypy.response.status = "200 OK: Everything worked as expected"

        return response

In [11]:
#endpoint /zone
class Zone(object):
    exposed = True

    def GET(self, *path, **query):

        #Check if the path is inserted
        if len(path) != 1:
            #If not return a Bad request error
            raise cherrypy.HTTPError(400, message='Bad Request: missing Zone name.')

        zone = path[0]
        
        from_ = int(query.get('from', -1))
        to_ = int(query.get('to', -1))


        #Check if the start time is inserted
        if int(from_) < 0:
            #Return a Bad request error
            raise cherrypy.HTTPError(400, 'Bad Request: missing start time.')
        
        #Check if the end time is inserted
        if  int(to_) < 0:
            #Return a Bad request error
            raise cherrypy.HTTPError(400, 'Bad Request: missing end time.')

        print(type(zone))
        try:
            print(zone)
            zone_values = []
            for k, v in encoded_zones.items():
                if k == zone:
                    for mac in v:
                        if redis_client.exists(mac):
                            mac_values = redis_client.ts().range(mac, from_time=from_, to_time=to_)
                            zone_values.append(mac_values)

            print(zone_values)
            # power_values = redis_client.ts().range(power_key, from_time=from_, to_time=to_)
        except:
            #Return a Bad request error
            raise cherrypy.HTTPError(404, 'Not Found: Unregistered mac addres for this zone is detected.')
        
        
        response_dict = {
            'zone': zone,
            'timestamps': [list(map(lambda x: x[0], mac_values)) for mac_values in zone_values]
            'labels': [list(map(lambda x: x[1], mac_values)) for mac_values in zone_values]
        }
        response = json.dumps(response_dict)


        cherrypy.response.status = "200 OK: Everything worked as expected"
        return response

    def DELETE(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing zone name')
        zone = path[0]
        print(zone)
        for k, v in encoded_zones.items():
            if k == zone:
                for mac in v:

                    mac_found = redis_client.delete(mac)
                    
                    #Check if the item has been removed
                    if mac_found == 0:
                        raise cherrypy.HTTPError(404, 'Not Found: invalid zone name')


        return 



SyntaxError: invalid syntax (2787038119.py, line 49)

In [6]:
class Devices(object):
    exposed = True

    def GET(self, *path, **query):
        keys = redis_client.keys('*')
        items = set()

        for key in keys:
            mac_address = key.decode()
            
            items.add(mac_address)

        # Coordinates?
        response_dict = {
            'mac_addresses': list(items)
        }

        response = json.dumps(response_dict)

        cherrypy.response.status = "200 OK: Everything worked as expected"

        return response

In [7]:
class Device(object):
    exposed = True

    def GET(self, *path, **query):

        #Check if the path is inserted
        if len(path) != 1:
            #If not return a Bad request error
            raise cherrypy.HTTPError(400, message='Bad Request: missing Device name.')

        mac_address = path[0]
        
        from_ = query.get('from', -1)
        to_ = query.get('to', -1)

        #Check if the start time is inserted
        if int(from_) < 0:
            #Return a Bad request error
            raise cherrypy.HTTPError(400, 'Bad Request: missing start time.')
        
        #Check if the end time is inserted
        if  int(to_) < 0:
            #Return a Bad request error
            raise cherrypy.HTTPError(400, 'Bad Request: missing end time.')

        
        # zone_key = zone + ':label'
        # power_key = zone+':power'
        print(type(mac_address))
        try:
            mac_values = redis_client.ts().range(mac_address, from_time=from_, to_time=to_)
            
        except:
            #Return a Bad request error
            raise cherrypy.HTTPError(404, 'Not Found: invalid mac_address.')
        
        
        response_dict = {
            'mac_address': mac_address,
            'timestamps': list(map(lambda x: x[0], mac_values)),
            'labels': list(map(lambda x: x[1], mac_values))
        }
        response = json.dumps(response_dict)

        cherrypy.response.status = "200 OK: Everything worked as expected"
        return response

    def DELETE(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing device name')
        mac_address = path[0]
        #Try to remove the item
        #battery_key = mac_address+':battery'
        #power_key = mac_address+':power'
        print(mac_address)
        mac_found = redis_client.delete(mac_address)
        #power_found = redis_client.delete(power_key)

        #Check if the item has been removed
        if mac_found == 0:
            raise cherrypy.HTTPError(404, 'Not Found: invalid device name')

        return 


In [8]:
conf = {'/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}}
cherrypy.tree.mount(Zones(), '/zones', conf)
cherrypy.tree.mount(Zone(), '/zone', conf)
cherrypy.tree.mount(Devices(), '/devices', conf)
cherrypy.tree.mount(Device(), '/device', conf)
cherrypy.config.update({'server.socket_host': '0.0.0.0'})
cherrypy.config.update({'server.socket_port': 8080})
cherrypy.engine.start()
cherrypy.engine.block()

[13/Apr/2023:15:22:56] ENGINE Bus STARTING
[13/Apr/2023:15:22:56] ENGINE Started monitor thread 'Autoreloader'.
[13/Apr/2023:15:22:56] ENGINE Serving on http://0.0.0.0:8080
[13/Apr/2023:15:22:56] ENGINE Bus STARTED
127.0.0.1 - - [13/Apr/2023:15:23:06] "GET /devices HTTP/1.1" 200 55 "" "python-requests/2.28.1"
<class 'str'>
127.0.0.1 - - [13/Apr/2023:15:23:10] "GET /device/0xdca9047270e2?from=1681077600000&to=1681336800000 HTTP/1.1" 200 861 "" "python-requests/2.28.1"
<class 'str'>
127.0.0.1 - - [13/Apr/2023:15:23:10] "GET /device/0x19f57bb40d93?from=1681077600000&to=1681336800000 HTTP/1.1" 200 65 "" "python-requests/2.28.1"
127.0.0.1 - - [13/Apr/2023:15:24:10] "GET /devices HTTP/1.1" 200 55 "" "python-requests/2.28.1"
<class 'str'>
127.0.0.1 - - [13/Apr/2023:15:24:11] "GET /device/0xdca9047270e2?from=0&to=1681399447688 HTTP/1.1" 200 861 "" "python-requests/2.28.1"
<class 'str'>
127.0.0.1 - - [13/Apr/2023:15:24:11] "GET /device/0x19f57bb40d93?from=0&to=1681399447688 HTTP/1.1" 200 21821 

In [9]:
#redis_client.flushdb()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=251b70cc-64ed-4269-9639-75a0d233eb14' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>